### 0. Default Setting

#### Library & Path Setting

In [68]:
import chromadb
import pandas as pd
import numpy as np 
import os 
import torch
import torch.nn.functional as F
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel

In [2]:
default_path = os.getcwd()
data_path = os.path.join(default_path, '../data')

#### Load data

In [26]:
data = pd.read_csv(os.path.join(data_path, 'test_data', 'koalphaca_v11.csv'))
print(f'data 개수: {len(data)}')
data.head(3)

data 개수: 21155


,question,answer
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...
1,스웨터의 유래는 어디에서 시작되었나요?,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 고리는 얼음과 ...,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카..."


In [116]:
samp = data.sample(5000)
samp.reset_index(inplace=True, drop=True)
samp.head(3)

,question,answer
0,부가가치세 예정신고에 대해 빠르게 알고 싶습니다.\n저는 대학원생이고 오는 25일이...,"부가가치세 예정신고는 법인일 경우 4월25일과 10월25일에, 개인일 경우 신규사업..."
1,최초의 애니메이션은 무엇이었을까요? 그리고 애니메이션 발전의 역사에 대해서도 궁금합니다.,"최초의 애니메이션은 프랑스의 에밀 꼴이 만든 ""팡타스마고리(Pantomimes Lu..."
2,고주파와 저주파 중 어느 쪽이 덜 들릴까요? 그 이유는 무엇일까요?,파동은 주파수가 높을수록 산란성이 높아져 멀리 전달되지 못합니다. 따라서 고주파가 ...


In [117]:
question = samp.question.values.tolist()
question[:2]

['부가가치세 예정신고에 대해 빠르게 알고 싶습니다.\n저는 대학원생이고 오는 25일이 부가가치세 예정신고일이라고 들었습니다. 일전에 있던 회사에서 세무사에게 모든 것을 대어맡으니 이제부터 막막하네요. 혹시 세무사를 찾아가지 않아도 세금계산서와 도장만 있으면 신고가 가능한 건가요? 그리고 사무실에서 쓰는 전기요금, 임대료, 사무용품 등 경비매출전표와 신용카드매출전표는 어떻게 해야 할까요? 그리고, 왜 예정신고와 확정신고가 있는 건가요?',
 '최초의 애니메이션은 무엇이었을까요? 그리고 애니메이션 발전의 역사에 대해서도 궁금합니다.']

### 1. ChromaDB Setting

#### Conenct ChromaDB

In [118]:
chroma_client = chromadb.HttpClient(host='3.39.250.201', port=8000)

In [119]:
chroma_client.list_collections()

[Collection(name=kor_emb)]

#### Create Collection

In [120]:
# Embedding Function
default_ef = embedding_functions.DefaultEmbeddingFunction()
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="multi-qa-distilbert-dot-v1")

In [121]:
emb_collection = chroma_client.get_collection(name='kor_emb')

In [122]:
emb_collection

Collection(name=kor_emb)

#### Load Model

In [123]:
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [124]:
emb = model.encode(question[0])

In [125]:
np.shape(emb)

(384,)

In [126]:
emb = np.array(model.encode(question[0]))
np.shape(emb.reshape(1, -1)) # np.mean(np.array(emb), axis=1)

(1, 384)

In [127]:
np.mean(emb.reshape(1, -1), axis=1)

array([-0.00252998], dtype=float32)

In [128]:
doc = []; embeddings = []; metadata = []; ids = []

for idx in range(len(samp)):
    # try:   # token 길이가 특정 값 넘어가면 임베딩 오류 발생   (512 토큰 이상 텍스트 임베딩 불가) 
    emb = model.encode(samp.question[idx])
    # emb = emb.reshape(1, -1) 
    # embedding = np.mean(np.array(emb), axis=1)
    # embedding = embedding.squeeze()
    embeddings.append(list(map(float, emb)))
    # except:
    #    print(idx)
    #    continue 
    doc.append(samp.question[idx])
    ids.append(str(idx + 1))

In [129]:
np.shape(embeddings), np.shape(doc)

((5000, 384), (5000,))

In [130]:
np.shape(embeddings), np.shape(doc), np.shape(ids)

((5000, 384), (5000,), (5000,))

In [134]:
emb_collection.add(
    documents=doc,
    embeddings=list(embeddings),
    # metadatas=metadata,
    ids=ids
)

In [136]:
emb_collection.peek(2)

{'ids': ['1', '10'],
 'embeddings': [[0.008779450319707394,
   0.07461600005626678,
   0.0038231832440942526,
   -0.037466470152139664,
   0.019713539630174637,
   0.0010351758683100343,
   0.16310028731822968,
   -0.05064982920885086,
   0.045647427439689636,
   -0.014302942901849747,
   0.1023234874010086,
   -0.08978056907653809,
   0.08996978402137756,
   -0.02701234444975853,
   0.001066669705323875,
   -0.06203601509332657,
   -0.04510023072361946,
   0.004087416455149651,
   -0.003286752849817276,
   -0.04952503368258476,
   -0.07199247181415558,
   0.0328700877726078,
   0.03412719443440437,
   -0.04707580432295799,
   -0.04984891414642334,
   -0.017115022987127304,
   0.0894135907292366,
   -0.04298828914761543,
   -0.06454932689666748,
   -0.036133117973804474,
   0.001223515602760017,
   0.0022011648397892714,
   -0.0553860105574131,
   0.03292061761021614,
   -0.0398440808057785,
   0.041251033544540405,
   -0.013002261519432068,
   0.07021893560886383,
   0.035576473921537

In [140]:
txt = '최초의 애니메이션은 ?'
print(txt)

results = emb_collection.query(
    query_texts = txt, 
    n_results = 5,
    # include=['distances'],
    # where={"source": {"$eq": "closed_qa"}},
    # where_document={"$contains":"safe"}
)

results

최초의 애니메이션은 ?


{'ids': [['2309', '3341', '2736', '3406', '1613']],
 'distances': [[0.7564581632614136,
   0.7682268619537354,
   0.7768236398696899,
   0.7787466645240784,
   0.7847808003425598]],
 'embeddings': None,
 'metadatas': [[None, None, None, None, None]],
 'documents': [['울산에 까마귀가 많은 이유는 무엇일까요?',
   '왜 세계시간의 기준이 영국의 GMT인가요? 그 이유는 무엇일까요?',
   '설탕이 인체에 끼치는 유해성은 무엇인가요?',
   '다윈개구리는 수컷이 알을 입속에 넣고 기른다는 것이 사실인가요??',
   '사진에서 이러한 물결무늬는 왜 생기는 건가요? 해상도가 안 맞나요?']],
 'uris': None,
 'data': None}